In [1]:
import json
from collections import defaultdict
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt 
import utils
import plot_utils
from tqdm.notebook import tqdm, tqdm_notebook
tqdm_notebook.pandas()


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
def load_public_dataset(tournament='World_Cup'):
    """
    Load the json files with the matches, events, players and competitions
    
    Parameters
    ----------
    data_folder : str, optional
        the path to the folder where json files are stored.
        
    tournaments : list, optional
        the list of tournaments to load. 
        
    Returns
    -------
    tuple
        a tuple of four dictionaries, containing matches, events, players and competitions
        
    """
    # loading the matches and events data
    matches, events = {}, {}
    with open('events/events_%s.json' %tournament) as json_data:
        events = json.load(json_data)
    with open('matches/matches_%s.json' %tournament) as json_data:
        matches = json.load(json_data)
    
    match_id2events = defaultdict(list)
    match_id2match = defaultdict(dict)
    for event in events:
        match_id = event['matchId']
        match_id2events[match_id].append(event)
                                         
    for match in matches:
        match_id = match['wyId']
        match_id2match[match_id] = match

    # loading the players data
    with open('players.json') as json_data:
        players = json.load(json_data)
    
    player_id2player = defaultdict(dict)
    for player in players:
        player_id = player['wyId']
        player_id2player[player_id] = player
    
    # loading the competitions data
    teams={}
    with open('teams.json') as json_data:
        teams = json.load(json_data)
    team_id2team = defaultdict(dict)
    for team in teams:
        team_id = team['wyId']
        team_id2team[team_id] = team

    #Corrigindo o nome do time Francês para igualar ao 'events'
    team_id2team[3799]['name'] = 'Angers SCO'
    
    return match_id2match, match_id2events, player_id2player, team_id2team

In [3]:
def passing_networks_half(match_id=2057954):
    """
    Construct the passing networks of the teams in the match.
    
    Parameters
    ----------
    match_id : int, optional
        identifier of the match to plot
        
    Returns
    -------
    tuple
        the two constructed networks, as networkx objects.
    """
    
    # take the names of the two teams of the match
    match_label = match_id2match[match_id]['label']
    team1_name = match_label.split(' - ')[0].strip()
    team2_name = match_label.split(' - ')[1].split(',')[0].strip()

    # take all the events of the match
    match_events = list(match_id2events[match_id])
    match_events_df = pd.DataFrame(match_events)
    first_half_max_duration = np.max(match_events_df[match_events_df['matchPeriod'] == '1H']['eventSec'])
    # sum 1H time end to all the time in 2H
    for event in match_events:
        if event['matchPeriod'] == '2H':
            event['eventSec'] += first_half_max_duration

    team2pass2weight_1H = defaultdict(lambda: defaultdict(int))
    team2pass2weight_2H = defaultdict(lambda: defaultdict(int))

    for event, next_event, next_next_event in zip(match_events, match_events[1:], match_events[2:]):
        try:
            if event['matchPeriod'] == '1H':
                if event['eventName'] == 'Pass' and 1801 in [tag['id'] for tag in event['tags']]:
                    sender = player_id2player[event['playerId']]['shortName'].encode('ascii', 'strict').decode('unicode-escape')
                    # if the next event of from a player of the same team
                    if (next_event['teamId'] == event['teamId']) & (next_event['playerId'] != event['playerId']):
                        receiver = player_id2player[next_event['playerId']]['shortName'].encode('ascii', 'strict').decode('unicode-escape')
                        team2pass2weight_1H[team_id2team[event['teamId']]['name']][(sender, receiver)] += 1
                    elif (next_next_event['teamId'] == event['teamId']) & (next_next_event['playerId'] != event['playerId']):
                        receiver = player_id2player[next_next_event['playerId']]['shortName'].encode('ascii', 'strict').decode('unicode-escape')
                        team2pass2weight_1H[team_id2team[event['teamId']]['name']][(sender, receiver)] += 1
            else:
                if event['eventName'] == 'Pass' and 1801 in [tag['id'] for tag in event['tags']]:
                    sender = player_id2player[event['playerId']]['shortName'].encode('ascii', 'strict').decode('unicode-escape')
                    # if the next event of from a player of the same team
                    if (next_event['teamId'] == event['teamId']) & (next_event['playerId'] != event['playerId']):
                        receiver = player_id2player[next_event['playerId']]['shortName'].encode('ascii', 'strict').decode('unicode-escape')
                        team2pass2weight_2H[team_id2team[event['teamId']]['name']][(sender, receiver)] += 1
                    elif (next_next_event['teamId'] == event['teamId']) & (next_next_event['playerId'] != event['playerId']):
                        receiver = player_id2player[next_next_event['playerId']]['shortName'].encode('ascii', 'strict').decode('unicode-escape')
                        team2pass2weight_2H[team_id2team[event['teamId']]['name']][(sender, receiver)] += 1
        except KeyError:
            pass
    # crete networkx graphs
    G1_1H, G2_1H = nx.DiGraph(team=team1_name), nx.DiGraph(team=team2_name)
    G1_2H, G2_2H = nx.DiGraph(team=team1_name), nx.DiGraph(team=team2_name)
    for (sender, receiver), weight in team2pass2weight_1H[team1_name].items():
        G1_1H.add_edge(sender, receiver, weight=weight)
    for (sender, receiver), weight in team2pass2weight_1H[team2_name].items():
        G2_1H.add_edge(sender, receiver, weight=weight)
    for (sender, receiver), weight in team2pass2weight_2H[team1_name].items():
        G1_2H.add_edge(sender, receiver, weight=weight)
    for (sender, receiver), weight in team2pass2weight_2H[team2_name].items():
        G2_2H.add_edge(sender, receiver, weight=weight)        
    
    return G1_1H, G2_1H, G1_2H, G2_2H

In [4]:
def pith_pos_to_dict(data):

    aux = data[['playerId', 'y', 'x']]
    aux['nome'] = aux['playerId'].apply(lambda p: player_id2player[p]['shortName'].encode('ascii', 'strict').decode('unicode-escape'))

    res = dict(zip(aux['nome'], zip(aux['x'], aux['y'])))

    return res

In [5]:
def plot_passing_networks(G1, G2, df_pos_1H, df_pos_2H):
    """
    Plot the two passing networks in input.
    
    Parameters
    ----------
    G1 : networkx object
        the object representing the first network
        
    G2 : networkx object
        the object representing the second network
    """
    #TO DO:
    # ***** Plotar nodes e edges baseado no peso *****

    player_pos1 = pith_pos_to_dict(df_pos_1H)
    player_pos2 = pith_pos_to_dict(df_pos_2H)

    fig, ax1 = plot_utils.pitch()
    fig, ax2 = plot_utils.pitch()
    

    nome2degree = dict(G1.degree)
    nx.draw(G1, pos=player_pos1 ,nodelist=list(nome2degree.keys()), 
            node_size=[deg * 50 for deg in nome2degree.values()], 
            node_color='red', edge_color='black',
            with_labels=True, font_weight='bold', alpha=0.75, ax=ax1)
    ax1.set_title('Time 1')
    ax1.set_xlim([0, 100])
    ax1.set_ylim([0, 100])
    ax1.set_axis_on()

    
    nome2degree = dict(G2.degree)
    nx.draw(G2, pos=player_pos2, nodelist=list(nome2degree.keys()), 
            node_size=[deg * 50 for deg in nome2degree.values()],
            node_color='blue', edge_color='black',
            with_labels=True, font_weight='bold', alpha=0.75, ax=ax2)
    ax2.set_title('Time 2')
    ax2.set_xlim([0, 100])
    ax2.set_ylim([0, 100])
    ax2.set_axis_on()
    
    plt.figure(figsize=(4, 3))
    plt.show()

In [6]:
#degree_centrality
def degree_centrality(netlist, i):
    data_avg = []
    data_std = []
    data_min = []
    data_max = []
    for net in netlist.values():
        data_avg.append(np.mean(list(nx.degree_centrality(net[i]).values())))
        data_std.append(np.std(list(nx.degree_centrality(net[i]).values())))
        data_min.append(min(list(nx.degree_centrality(net[i]).values())))
        data_max.append(max(list(nx.degree_centrality(net[i]).values())))

    return pd.DataFrame(list(zip(data_avg, data_std, data_min, data_max)), columns=[f'avg_degree_centrality_T{i+1}', f'std_degree_centrality_T{i+1}', f'min_degree_centrality_T{i+1}', f'max_degree_centrality_T{i+1}'])


#betwenness_centrality
def betwenness_centrality(netlist, i):
    data_avg = []
    data_std = []
    data_min = []
    data_max = []
    for net in netlist.values():
        data_avg.append(np.mean(list(nx.betweenness_centrality(net[i]).values())))
        data_std.append(np.std(list(nx.betweenness_centrality(net[i]).values())))
        data_min.append(min(list(nx.betweenness_centrality(net[i]).values())))
        data_max.append(max(list(nx.betweenness_centrality(net[i]).values())))

    return pd.DataFrame(list(zip(data_avg, data_std, data_min, data_max)), columns=[f'avg_betweenness_centrality_T{i+1}', f'std_betweenness_centrality_T{i+1}', f'min_betweenness_centrality_T{i+1}', f'max_betweenness_centrality_T{i+1}'])

#closeness_centrality
def closeness_centrality(netlist, i):
    data_avg = []
    data_std = []
    data_min = []
    data_max = [] 
    for net in netlist.values():
        data_avg.append(np.mean(list(nx.closeness_centrality(net[i]).values())))
        data_std.append(np.std(list(nx.closeness_centrality(net[i]).values())))
        data_min.append(min(list(nx.closeness_centrality(net[i]).values())))
        data_max.append(max(list(nx.closeness_centrality(net[i]).values())))

    return pd.DataFrame(list(zip(data_avg, data_std, data_min, data_max)), columns=[f'avg_closeness_centrality_T{i+1}', f'std_closeness_centrality_T{i+1}', f'min_closeness_centrality_T{i+1}', f'max_closeness_centrality_T{i+1}'])

#eigenvalue_centrality
def eigenvector_centrality(netlist, i):
    data_avg = []
    data_std = []
    data_min = []
    data_max = []
    for net in netlist.values():
        data_avg.append(np.mean(list(nx.eigenvector_centrality(net[i]).values())))
        data_std.append(np.std(list(nx.eigenvector_centrality(net[i]).values())))
        data_min.append(min(list(nx.eigenvector_centrality(net[i]).values())))
        data_max.append(max(list(nx.eigenvector_centrality(net[i]).values())))

    return pd.DataFrame(list(zip(data_avg, data_std, data_min, data_max)), columns=[f'avg_eigenvector_centrality_T{i+1}', f'std_eigenvector_centrality_T{i+1}', f'min_eigenvector_centrality_T{i+1}', f'max_eigenvector_centrality_T{i+1}'])

#clustering coefficient
def clustering_coefficient(netlist, i):
    data_avg = []
    data_std = []
    data_min = []
    data_max = []
    for net in netlist.values():
        data_avg.append(np.mean(list(nx.clustering(net[i]).values())))
        data_std.append(np.std(list(nx.clustering(net[i]).values())))
        data_min.append(min(list(nx.clustering(net[i]).values())))
        data_max.append(max(list(nx.clustering(net[i]).values())))

    return pd.DataFrame(list(zip(data_avg, data_std, data_min, data_max)), columns=[f'avg_clustering_T{i+1}', f'std_clustering_T{i+1}', f'min_clustering_T{i+1}', f'max_clustering_T{i+1}'])

#average shortest path lenght
def avg_shortest_path(netlist, i):
    data_avg = []
    for net in netlist.values():
        data_avg.append((nx.average_shortest_path_length(net[i])))

    return pd.DataFrame(data_avg, columns=[f'avg_shortest_path_T{i+1}'])

#position centroid
def centroid(netlist, pitch_coords, i):
    centroid = pitch_coords.groupby(['matchId', 'teamId']).agg({'y':([np.mean, np.std]), 'x':([np.mean, np.std])}).reset_index()
    centroid.columns = ['matchId', 'teamId', f'mean_centroid_y_T{i+1}', f'std_centroid_y_T{i+1}', f'mean_centroid_x_T{i+1}', f'std_centroid_x_T{i+1}']
    
    return centroid

In [7]:
for tournament in tqdm(utils.TOURNAMENTS, total= len(utils.TOURNAMENTS)):
    print("TOURNAMENT: %s" %tournament)

    # carrega dados
    print("carregando dados")
    match_id2match, match_id2events, player_id2player, team_id2team = load_public_dataset(tournament)
    matches = pd.read_json('matches\matches_%s.json' %tournament)
    events = pd.read_json('events\events_%s.json' %tournament)

    #consertando erros do dataset
    if tournament == 'France':
        matches.loc[1, 'winner'] = 3804
        matches.loc[37, 'winner'] = 3770
    elif tournament == 'Germany':
        matches.loc[48, 'winner'] = 2450
        matches.loc[102, 'winner'] = 2449

    #tira penalidades
    events = events[events['matchPeriod'] != 'P']

    # converte coluna 'tags' par algo manipulável
    print("convertendo tags")
    events['tags'] = events['tags'].map(utils.converting_tags)

    # gera coordenada média de cada node
    print("gerando coordenadas médias")
    # gera coordenada média de cada node - 1H
    passes_1H = events[(events['eventName'] == 'Pass') & (events['matchPeriod'] == '1H') & (events['tags'].apply(lambda x: (1801 in x)))]

    coords_1H = passes_1H['positions'].apply(pd.Series)[0].apply(pd.Series)
    passes_1H = passes_1H.join(coords_1H)
    passes_1H = passes_1H.drop(['positions'], axis = 1)

    pitch_coords_1H = passes_1H.groupby(['matchId', 'teamId', 'playerId']).agg(y=('y','mean'), x=('x','mean'))
    pitch_coords_1H = pitch_coords_1H.reset_index()

    # gera coordenada média de cada node - 2H
    passes_2H = events[(events['eventName'] == 'Pass') & (events['matchPeriod'] != '1H') & (events['tags'].apply(lambda x: (1801 in x)))]

    coords_2H = passes_2H['positions'].apply(pd.Series)[0].apply(pd.Series)
    passes_2H = passes_2H.join(coords_2H)
    passes_2H = passes_2H.drop(['positions'], axis = 1)

    pitch_coords_2H = passes_2H.groupby(['matchId', 'teamId', 'playerId']).agg(y=('y','mean'), x=('x','mean'))
    pitch_coords_2H = pitch_coords_2H.reset_index()

    #gera redes
    net_list_1H = defaultdict(tuple)
    net_list_2H = defaultdict(tuple)
    for matchID in match_id2match.keys():    
        G1, G2, G3, G4 = passing_networks_half(match_id = matchID)
        par_1H = (G1, G2)
        par_2H = (G3, G4)
        net_list_1H[matchID] = par_1H
        net_list_2H[matchID] = par_2H


    # - Gera bases -
    print("gerando base")

    # -- HOME --

    # 1 HALF

    dfh_1h = matches[['wyId']]
    dfh_1h = dfh_1h.rename(columns={'wyId':'matchID'})

    home_l = []
    away_l = []

    for match in matches['teamsData']:
        t0 = list(match.values())[0]
        t1 = list(match.values())[1]

        if t0['side'] == 'home':
            home_l.append(t0['teamId'])
            away_l.append(t1['teamId'])  
        else: 
            away_l.append(t0['teamId'])
            home_l.append(t1['teamId'])

    dfh_1h['team1_ID'] = home_l
    dfh_1h['team2_ID'] = away_l

    dfh_1h['date'] = matches['date']

    dfh_1h['tournament'] = tournament

    closeness = closeness_centrality(net_list_1H, 0)
    degree = degree_centrality(net_list_1H, 0)
    betwenness = betwenness_centrality(net_list_1H, 0)
    eigenvector = eigenvector_centrality(net_list_1H, 0)
    clustering = clustering_coefficient(net_list_1H, 0)
    shortest_path = avg_shortest_path(net_list_1H, 0)
    df_centroid = centroid(net_list_1H, pitch_coords_1H, 0)

    dfh_1h = dfh_1h.join(closeness)
    dfh_1h = dfh_1h.join(degree)
    dfh_1h = dfh_1h.join(betwenness)
    dfh_1h = dfh_1h.join(eigenvector)
    dfh_1h = dfh_1h.join(clustering)
    dfh_1h = dfh_1h.join(shortest_path)

    dfh_1h = dfh_1h.merge(df_centroid, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    # 2 HALF

    dfh_2h = matches[['wyId']]
    dfh_2h = dfh_2h.rename(columns={'wyId':'matchID'})

    home_l = []
    away_l = []

    for match in matches['teamsData']:
        t0 = list(match.values())[0]
        t1 = list(match.values())[1]

        if t0['side'] == 'home':
            home_l.append(t0['teamId'])
            away_l.append(t1['teamId'])  
        else: 
            away_l.append(t0['teamId'])
            home_l.append(t1['teamId'])

    dfh_2h['team1_ID'] = home_l
    dfh_2h['team2_ID'] = away_l

    dfh_2h['date'] = matches['date']

    dfh_2h['tournament'] = tournament

    closeness = closeness_centrality(net_list_2H, 0)
    degree = degree_centrality(net_list_2H, 0)
    betwenness = betwenness_centrality(net_list_2H, 0)
    eigenvector = eigenvector_centrality(net_list_2H, 0)
    clustering = clustering_coefficient(net_list_2H, 0)
    shortest_path = avg_shortest_path(net_list_2H, 0)
    df_centroid = centroid(net_list_2H, pitch_coords_2H, 0)

    dfh_2h = dfh_2h.join(closeness)
    dfh_2h = dfh_2h.join(degree)
    dfh_2h = dfh_2h.join(betwenness)
    dfh_2h = dfh_2h.join(eigenvector)
    dfh_2h = dfh_2h.join(clustering)
    dfh_2h = dfh_2h.join(shortest_path)

    dfh_2h = dfh_2h.merge(df_centroid, how='left', left_on=['matchID', 'team1_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    # -- AWAY --

    # 1 HALF

    dfa_1h = matches[['wyId']]
    dfa_1h = dfa_1h.rename(columns={'wyId':'matchID'})

    dfa_1h['team1_ID'] = home_l
    dfa_1h['team2_ID'] = away_l

    dfa_1h['date'] = matches['date']

    dfa_1h['tournament'] = tournament

    closeness = closeness_centrality(net_list_1H, 1)
    degree = degree_centrality(net_list_1H, 1)
    betwenness = betwenness_centrality(net_list_1H, 1)
    eigenvector = eigenvector_centrality(net_list_1H, 1)
    clustering = clustering_coefficient(net_list_1H, 1)
    shortest_path = avg_shortest_path(net_list_1H, 1)
    df_centroid = centroid(net_list_1H, pitch_coords_1H, 1)

    dfa_1h = dfa_1h.join(closeness)
    dfa_1h = dfa_1h.join(degree)
    dfa_1h = dfa_1h.join(betwenness)
    dfa_1h = dfa_1h.join(eigenvector)
    dfa_1h = dfa_1h.join(clustering)
    dfa_1h = dfa_1h.join(shortest_path)

    dfa_1h = dfa_1h.merge(df_centroid, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)

    # 2 HALF

    dfa_2h = matches[['wyId']]
    dfa_2h = dfa_2h.rename(columns={'wyId':'matchID'})

    dfa_2h['team1_ID'] = home_l
    dfa_2h['team2_ID'] = away_l

    dfa_2h['date'] = matches['date']

    dfa_2h['tournament'] = tournament

    closeness = closeness_centrality(net_list_2H, 1)
    degree = degree_centrality(net_list_2H, 1)
    betwenness = betwenness_centrality(net_list_2H, 1)
    eigenvector = eigenvector_centrality(net_list_2H, 1)
    clustering = clustering_coefficient(net_list_2H, 1)
    shortest_path = avg_shortest_path(net_list_2H, 1)
    df_centroid = centroid(net_list_2H, pitch_coords_2H, 1)

    dfa_2h = dfa_2h.join(closeness)
    dfa_2h = dfa_2h.join(degree)
    dfa_2h = dfa_2h.join(betwenness)
    dfa_2h = dfa_2h.join(eigenvector)
    dfa_2h = dfa_2h.join(clustering)
    dfa_2h = dfa_2h.join(shortest_path)

    dfa_2h = dfa_2h.merge(df_centroid, how='left', left_on=['matchID', 'team2_ID'], right_on=['matchId', 'teamId']).drop(['matchId', 'teamId'], axis=1)


    #definindo vencedor e retirando empates apenas do dfh_1h (df home)
    dfh_1h['winner'] = matches['winner']
    dfh_1h = dfh_1h.loc[dfh_1h['winner'] != 0]
    dfh_1h['winner'] = np.where((dfh_1h['winner'] == dfh_1h['team1_ID']), 1, 0)
    #definindo vencedor e retirando empates apenas do dfh_1h (df home)
    dfh_2h['winner'] = matches['winner']
    dfh_2h = dfh_2h.loc[dfh_2h['winner'] != 0]
    dfh_2h['winner'] = np.where((dfh_2h['winner'] == dfh_2h['team1_ID']), 1, 0)

    dfa_1h = dfa_1h.loc[matches['winner'] != 0]
    dfa_2h = dfa_2h.loc[matches['winner'] != 0]


    print("gerando csv")
    dfh_1h.to_csv('analitics/network_%s_home_1h.csv' %tournament, encoding='utf-8', index=False)
    dfa_1h.to_csv('analitics/network_%s_away_1h.csv' %tournament, encoding='utf-8', index=False)
    dfh_2h.to_csv('analitics/network_%s_home_2h.csv' %tournament, encoding='utf-8', index=False)
    dfa_2h.to_csv('analitics/network_%s_away_2h.csv' %tournament, encoding='utf-8', index=False)

    print('-----------------')

  0%|          | 0/7 [00:00<?, ?it/s]

TOURNAMENT: Italy
carregando dados
convertendo tags
gerando coordenadas médias
gerando base
gerando csv
-----------------
TOURNAMENT: England
carregando dados
convertendo tags
gerando coordenadas médias
gerando base
gerando csv
-----------------
TOURNAMENT: Germany
carregando dados
convertendo tags
gerando coordenadas médias
gerando base
gerando csv
-----------------
TOURNAMENT: France
carregando dados
convertendo tags
gerando coordenadas médias
gerando base
gerando csv
-----------------
TOURNAMENT: Spain
carregando dados
convertendo tags
gerando coordenadas médias
gerando base
gerando csv
-----------------
TOURNAMENT: European_Championship
carregando dados
convertendo tags
gerando coordenadas médias
gerando base
gerando csv
-----------------
TOURNAMENT: World_Cup
carregando dados
convertendo tags
gerando coordenadas médias
gerando base
gerando csv
-----------------


In [13]:
dfh_1h

,matchID,team1_ID,team2_ID,date,tournament,avg_closeness_centrality_T1,std_closeness_centrality_T1,min_closeness_centrality_T1,max_closeness_centrality_T1,avg_degree_centrality_T1,std_degree_centrality_T1,min_degree_centrality_T1,max_degree_centrality_T1,avg_betweenness_centrality_T1,std_betweenness_centrality_T1,min_betweenness_centrality_T1,max_betweenness_centrality_T1,avg_eigenvector_centrality_T1,std_eigenvector_centrality_T1,min_eigenvector_centrality_T1,max_eigenvector_centrality_T1,avg_clustering_T1,std_clustering_T1,min_clustering_T1,max_clustering_T1,avg_shortest_path_T1,mean_centroid_y_T1,std_centroid_y_T1,mean_centroid_x_T1,std_centroid_x_T1,winner
0,2058017,4418,9598,2018-07-15 17:00:00-02:00,World_Cup,0.641414,0.033406,0.555556,0.666667,0.927273,0.195824,0.6,1.3,0.062626,0.045716,0.006481,0.189444,0.298792,0.040401,0.203641,0.343949,0.481299,0.073976,0.365385,0.615385,1.563636,40.772290,26.616420,40.213104,13.761709,1
1,2058016,5629,2413,2018-07-14 16:00:00-02:00,World_Cup,0.763464,0.062436,0.666667,0.909091,1.363636,0.256808,0.8,1.7,0.035354,0.015776,0.009365,0.065370,0.298410,0.043131,0.211039,0.385501,0.700788,0.037793,0.632743,0.764706,1.318182,47.011325,22.961150,49.758691,16.213447,1
2,2058015,9598,2413,2018-07-11 20:00:00-02:00,World_Cup,0.706864,0.066058,0.588235,0.833333,1.145455,0.284009,0.7,1.5,0.047475,0.031702,0.011111,0.132778,0.294749,0.063498,0.159114,0.377917,0.626738,0.061811,0.511628,0.700000,1.427273,46.535838,28.466312,47.196026,18.031903,1
3,2058014,4418,5629,2018-07-10 20:00:00-02:00,World_Cup,0.712496,0.083712,0.500000,0.833333,1.181818,0.269066,0.5,1.6,0.047475,0.030361,0.002778,0.122407,0.291499,0.077055,0.087153,0.394158,0.636729,0.076694,0.507937,0.777778,1.427273,47.433662,25.094532,49.415909,16.746089,1
4,2058012,14358,9598,2018-07-07 20:00:00-02:00,World_Cup,0.681346,0.084823,0.555556,0.833333,1.054545,0.231059,0.6,1.5,0.054545,0.027290,0.007407,0.099378,0.287291,0.091504,0.132618,0.426848,0.563122,0.063740,0.460784,0.727273,1.490909,48.430866,20.363275,42.282246,17.162265,0
5,2058013,7047,2413,2018-07-07 16:00:00-02:00,World_Cup,0.755638,0.075884,0.666667,0.909091,1.327273,0.191125,1.0,1.6,0.037374,0.015011,0.012712,0.067341,0.296533,0.054567,0.220754,0.402187,0.657603,0.043544,0.582192,0.730769,1.336364,48.344010,23.463136,37.908240,10.791786,0
6,2058011,6380,5629,2018-07-06 20:00:00-02:00,World_Cup,0.697773,0.118329,0.400000,0.833333,1.127273,0.417984,0.2,1.7,0.054545,0.059158,0.000000,0.202593,0.283014,0.103981,0.025510,0.398430,0.759940,0.109240,0.558140,1.000000,1.490909,49.639858,25.022814,50.262538,18.946653,0
7,2058010,15670,4418,2018-07-06 16:00:00-02:00,World_Cup,0.689423,0.111123,0.416667,0.833333,1.090909,0.264419,0.5,1.3,0.055556,0.037859,0.000000,0.146111,0.281680,0.107543,0.044805,0.443040,0.583324,0.077311,0.500000,0.777778,1.500000,53.460870,18.723267,46.833930,16.577338,0
8,2058009,12430,2413,2018-07-03 20:00:00-02:00,World_Cup,0.740376,0.091317,0.555556,0.909091,1.254545,0.336732,0.5,1.6,0.041414,0.029392,0.006587,0.107593,0.290131,0.082056,0.093882,0.405563,0.682301,0.066335,0.575342,0.819672,1.372727,51.910101,23.578475,44.205978,15.498467,0
9,2058008,7047,6697,2018-07-03 16:00:00-02:00,World_Cup,0.689964,0.069254,0.588235,0.833333,1.072727,0.195824,0.8,1.3,0.051515,0.034697,0.009259,0.125926,0.291484,0.077110,0.156593,0.422249,0.569245,0.075707,0.430556,0.750000,1.463636,48.262699,21.786132,44.977207,16.527349,1


In [9]:
first_net = next(iter(net_list))
plot_passing_networks(net_list[first_net][0], net_list[first_net][1], pitch_coords[pitch_coords['matchId'] == first_net])


NameError: name 'net_list' is not defined